UNDER CONSTRUCTION

<div class="container">
   <div style="float:left;width:20%"><a href="./Ingestion.ipynb"><< Section 2: Data Ingestion</a></div>
   <div style="float:right;width:20%"><a href="./Model.ipynb">Section 4: Model Creation >></a></div>
   <div style="float:right;width:40%"><a href="../main.md">Table of Contents</a></div>
</div>